In [1]:
import sys
import struct
import redis
import numpy as np

sys.path.append('../')

In [2]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [3]:
array = np.ones((2, 3))
array

array([[1., 1., 1.],
       [1., 1., 1.]])

In [4]:
def toRedis(r,a,n):
   """Store given Numpy array 'a' in Redis under key 'n'"""
   h, w = a.shape
   shape = struct.pack('>II',h,w)
   encoded = shape + a.tobytes()
   
   # Store encoded data in Redis
   r.set(n,encoded)
   return

def fromRedis(r,n):
   """Retrieve Numpy array from Redis key 'n'"""
   encoded = r.get(n)
   h, w = struct.unpack('>II',encoded[:8])
   # Add slicing here, or else the array would differ from the original
   a = np.frombuffer(encoded[8:]).reshape(h,w)
   return a

In [10]:
# Create 80x80 numpy array to store
a0 = np.arange(6400,dtype=np.float64).reshape(80,80) 
a0

array([[0.000e+00, 1.000e+00, 2.000e+00, ..., 7.700e+01, 7.800e+01,
        7.900e+01],
       [8.000e+01, 8.100e+01, 8.200e+01, ..., 1.570e+02, 1.580e+02,
        1.590e+02],
       [1.600e+02, 1.610e+02, 1.620e+02, ..., 2.370e+02, 2.380e+02,
        2.390e+02],
       ...,
       [6.160e+03, 6.161e+03, 6.162e+03, ..., 6.237e+03, 6.238e+03,
        6.239e+03],
       [6.240e+03, 6.241e+03, 6.242e+03, ..., 6.317e+03, 6.318e+03,
        6.319e+03],
       [6.320e+03, 6.321e+03, 6.322e+03, ..., 6.397e+03, 6.398e+03,
        6.399e+03]])

In [11]:
# Store array a0 in Redis under name 'a0array'
toRedis(r,a0,'a0array')

In [12]:
aux = r.get('a0array')
type(aux)

bytes

In [14]:
# Retrieve from Redis
a1 = fromRedis(r,'a0array')


In [15]:
np.testing.assert_array_equal(a0,a1)